# Analysing artisanal small-scale mining in the DRC between 2013 and 2022 via NDVI and Sentinel 2A imagery 

This code analyses changes in artisanal small-scale (ASM) mining activity at an illegal mining site in Bisie in the DR Congo (DRC) between the years 2016 and 2022. While ASM provides a livelihood to more than 45m peoples globally, it also is associated with financing military conflict groups and with environmental degradation, such as deforestation and mercury pollution. 

This code calculates the NDVI for Sentinel 2A imagery as well as the change of the NDVI between the years 2016 and 2022. Prior to the NDVI calculation, the code first loads and reads the imagery into memory, inspects several key characteristics of the imagery, and displays the False Colour Composite of the imagery for both years.

First, we start by importing the necessary modules:


In [3]:
%matplotlib notebook

import numpy as np
import rasterio as rio
from rasterio import plot 
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from rasterio.plot import show


Now, we open the datasets and load the imageries into the memory. We load all bands of the imagery, that is, band1(blue), band2(green), band3(red) and band8(NIR). If you were to load only one particular band, you would need to specify the number of the band in the read() method. For instance, for loading only band8(NIR), you would need to modify the code as, e.g. img16 = dataset16.read(3). Be careful: in the read() method, indices start from 1, not from 0. 

In [5]:
#Opens the files from 2016 and 2022
dataset16 = rio.open("D://RS_Ulster//sub_16.tif")
dataset22 = rio.open("D://RS_Ulster//sub_22.tif")

In [6]:
#loads the raster imageries into memory; 
img16 = dataset16.read()
img22 = dataset22.read()

Next, we verify the metadata as well as some of the attribues from the datasets and loaded images. 

In [ ]:
#verifies the datasets´ meta-data, including the driver, datatype, width, height, crs and layer count of the dataset
print('the 2016 dataset´s metadata is:  {}'.format(dataset16.meta))
print('the 2022 dataset´s metadata is:  {}'.format(dataset22.meta))

#alternatively, we could also verify these informations via the following methods, such as, for instance:
dataset16.height #number of columns
dataset16.width #number of rows
dataset16.crs #coordinate reference system
dataset16.count #number of layers/bands



In [ ]:
#verifies the datasets´ Bounding Box
print('for 2016: {}'.format(dataset16.bounds))
print('for 2022: {}'.format(dataset22.bounds))


The loaded images consist of arrays with the pixel values of the particular bands. Similar to lists and tuples, these arrays can be indexed. If there are more than just one band, the first index corresponds to the band, the second to the row (y) location, and the third to the column (x). If there is only one band, the first would correspond to the row (y) location, the second to the column (x) location. 

In [ ]:
print(img22[2]) #gets the array of pixel values for band 3 (red) for image 2022
print(img22[2,444,344]) #gets the pixel values for band 3 (red) for row (x) of 444 and column (y) of 344 
print(img22[3,444,344]) #gets the pixel values for band 4 (NIR) for row (x) of 444 and column (y) of 344 
print(img22[2:,444, 344])  #gets the pixel values for band 3(red) and 4 (NIR) for row (x) of 444 and column (y) of 344 

In the next line, we verify the shape of the loaded imagery and verify if the shapes are the same for both bands. This is important because if the shapes were not the same, we could not perform calculations between the arrays. 

In [ ]:
print(img16.shape, img22.shape) #gets the shapes for both images 
img16.shape == img22.shape #verifies if both images have the same shape

In the following line, we verify both the maximum and minimum pixel value of all bands for both images. 

In [ ]:
#verifies the max pixel values for both images for all bands 
print(np.max(img16, axis=(1, 2)), np.min(img16, axis=(1, 2)))
print(np.max(img22, axis=(1, 2)), np.min(img22, axis=(1, 2)))

#alternatively, we could do it like that:
maxvals = [img16[i].max() for i in range(dataset16.count)]
print(maxvals)
minvals = [img16[i].min() for i in range(dataset16.count)]
print(minvals)

Now, we create a figure to plot our imagery. First, we define the CRS for both images for cartopy. As we have seen with the csr(method), both images are from EPSG 32735. In this link we can verify which coordinate reference system the EPSG corresponds to: https://epsg.io/ . Then, we set the boundaries for the imagery, for which we can use the bounds-methods. The Bounding Box is for both images the same. Finally, we establish the axes and figure size for our plot. 

In [ ]:
myCRS = ccrs.UTM(35) 
xmin, ymin, xmax, ymax = dataset22.bounds #as both images have the same bounds we can just pick one year
fig, ax = plt.subplots(1, 1, figsize=(5, 5), subplot_kw=dict(projection=myCRS))

In this function, we address two problems: 
First, the read()methods loads the raster in a way in which: bands = index 1, rows = index 2, columns = index 3. However, imshow () needs the indices to be order as:  rows = index 1, columns = index 2, bands = index 3. So we need to re-order the indices by applying the tranpose() method. 
Secondly, supported array values for more than three bands must be either between 0-1 float or 0-255int. Thus, we need to also re-scale our image to have values between 0-1 (or 0-255). We do this simply by dividing our pixel values / 10000 and transforming the dtype into float. If we would have wanted the 0 -255 option, we could have also applied the following code:(255 dispimg = (255 *(img / 10000)).astype(np.uint8) 

In [ ]:
def img_display(image, ax, bands, transform, extent):
    '''
    This function first re-orders the indices by applying the transpose()method
    Then it re-scales to values between 0 and 1 and changes the dtype to float. 
    Finally, it creates the handle to display the image with the ax.imshow() method.
    
    The parameters of the img_display function are the following: 
    
    Image: loaded imagery
    ax: axes
    bands: bands you want to have displayed
    extent: extent/boundaries of the imagery
    '''
    # first, we transpose the image to re-order the indices
    dispimg = image.transpose([1, 2, 0])
    
    # next, we have to scale the image.
    dispimg = (dispimg / 10000).astype(np.float64)
    
    # finally, we display the image
    handle = ax.imshow(dispimg[:, :, bands], transform=transform, extent=extent)
    
    
    return handle, ax

The next line calls the function img_display as a False Colour Composite (NIR, red, green). If you would like to display a True Colour Composite, you need to change the band numbers from [3, 2, 1] to [2, 1, 0].  The code also saves the figure as "map1". You can find that map1 then in your Jupyter Notebook. To display and save the imagery of year 2022, you need to change the first argument from "img16" to "img22". To save the figure for year 2022, you also need to change "map1.png" to "map2.pgn" -- otherwise it just overwrites the figure from year 2016. 

In [ ]:
h, ax = img_display(img16, ax, [3,2,1], myCRS, [xmin, xmax, ymin, ymax])
fig 
fig.savefig('map1.png', bbox_inches='tight', dpi=200)

As you will notice, while the displaying of the images serves a first impression, both images are quite dark. That is, to see a bit more detail, we have to perform a percentile stretch so that images are displayed in a brighter way. 

In [ ]:
def percentile_stretch(image, pmin=0., pmax=100.):
    '''
    This function calculates a percentile stretch for the imagery. 
    First, it makes sure that that pmin < pmax, and that they are between 0, 100 and assures that the image is only 2-dimensional
    Secondly, it defines the minval and maxval
    Finally, we calculate the stretch and make sure that all values below the minimum  are set to 0 and the ones above maxval to 1. 
    
    The parameters for the function are: 
    image = imagery that you want to display
    pmin, pmax = the min and max of the values to be displayed. percentile values are between 0 and 100.  
    '''
    # setting pmin < pmax to values between 0, 100
    if not 0 <= pmin < pmax <= 100:
        raise ValueError('0 <= pmin < pmax <= 100')
    # raising an error if image is not 2-dimensional
    if not image.ndim == 2:
        raise ValueError('Image can only have two dimensions (row, column)')
    
    minval = np.percentile(image, pmin)
    maxval = np.percentile(image, pmax)
    
    stretched = (image - minval) / (maxval - minval) # stretch the image to 0, 1
    stretched[image < minval] = 0 # set anything less than minval to the new minimum, 0.
    stretched[image > maxval] = 1 # set anything greater than maxval to the new maximum, 1.
    
    return stretched

In [ ]:
def new_img_display(image, ax, bands, stretch_args=None, **imshow_kwargs):
    '''
    This function first copies the original image, setting the dtype as float. 
    Secondly, the function applies the percentile_stretch_function. 
    Finally, indices are re-ordered via the transpose()method 
    
    The parameters of the img_display function are the following: 
    
    Image: loaded imagery
    ax: axes
    bands: bands you want to have displayed
    extent & transform are epxressed as args (None) and **kwargs
    '''
    dispimg = image.copy().astype(np.float32) # make a copy of the original image,
    # but be sure to cast it as a floating-point image, rather than an integer

    for b in range(image.shape[0]): # loop over each band, stretching using percentile_stretch()
        if stretch_args is None: # if stretch_args is None, use the default values for percentile_stretch
            dispimg[b] = percentile_stretch(image[b])
        else:
            dispimg[b] = percentile_stretch(image[b], **stretch_args)

    # next, we transpose the image to re-order the indices
    dispimg = dispimg.transpose([1, 2, 0])
    
    # finally, we display the image
    handle = ax.imshow(dispimg[:, :, bands], **imshow_args)
    
    return handle, ax

In [ ]:
my_kwargs = {'extent': [xmin, xmax, ymin, ymax],
             'transform': myCRS}

my_stretch = {'pmin': 0.1, 'pmax': 99.9}



The next line calls the function new_img_display for a False Colour composite (Nir, red, green) and saves the figure as "map3.pgn" For displaying a True Colour composite, you need to change the bands to [2,1,0]. 
For displaying the imagery for year 2022, you need to change the first paramter of the function from "img16" to "img22". Again, remember to also change the name of the map before saving the figure for year 2022.  

In [ ]:
h, ax = new_img_display(img16, ax, [3, 2, 1], stretch_args=my_stretch, **my_kwargs)
fig
fig.savefig('map2.png', bbox_inches='tight', dpi=200)


Now that we have inspected both images visually, we can perform our NDVI calculation. As a first step, we have to re-scale both images to make sure the pixel values are between 0 and 1 as well as floats (fractions).

In [1]:
#scales the images to a value between 0 and 1
img16_scaled = (img16/10000).astype(np.float64)
img22_scaled = (img22/10000).astype(np.float64)

img16_scaled.dtype




NameError: name 'img16' is not defined

In [ ]:
print(img16_scaled[3]) #displays the array with the re-scaled pixel values for band 4(NIR) of year 2016

Now, we calculate the NDVI for both years. To do so, we apply the formula NDVI = (NIR) -(Red) / (NIR) + (Red) for our code. 


In [ ]:
ndvi16 = (img16_scaled[3]-img16_scaled[2]) / (img16_scaled[3] + img16_scaled[2]) #calculates the NDVI ()
ndvi22 = (img22_scaled[3]-img22_scaled[2]) / (img22_scaled[3] + img22_scaled[2]) #calculates the NDVI ()



The next lines of code display the NDVI imagery. We have set the vmin and vmax to -1 and 1 as the NDVI values ranges between -1 and 1. To display the NDVI for year 2022, exchange the first argument of ax.imshow() from "ndvi16" to "ndvi22". Make sure you also exchange the name of the map when you save the figure. 

In [ ]:
myCRS = ccrs.UTM(35) 
xmin, ymin, xmax, ymax = dataset22.bounds #as both images have the same bounds we can just pick one year
fig, ax = plt.subplots(1, 1, figsize=(5, 5), subplot_kw=dict(projection=myCRS))

In [ ]:
ax.imshow(ndvi16, cmap="brg",vmin=-1, vmax=1, transform=myCRS, extent=[xmin, xmax, ymin, ymax]) #vmin and vmax bc NDVI is between -1 and 1
fig 
fig.savefig('map3.png', bbox_inches='tight', dpi=200)

Finally, we calculate the difference of the NDVI between year 2016 and 2022. 

In [ ]:
ndvi_diff = ndvi16 - ndvi22
print(ndvi_diff.min(), ndvi_diff.max())

In [ ]:
ax.imshow(ndvi_diff, cmap="rainbow",vmin=-1, vmax=1, transform=myCRS, extent=[xmin, xmax, ymin, ymax]) #displays the NDVI difference
fig
fig.savefig('map6.png', bbox_inches='tight', dpi=200)

CONGRATULATIONS! You have reached the end of my script and can now enjoy the sunny weather outside. 

PS: Several quibbeling remained that I sadly could not solve. 
First, I was wondering how I could make a function so that I can always apply methods to both images: img16 and img22. I thought to make a list of the datasets (see function below). Making a list of the loaded images did not solve the problem either. 


In [ ]:
#dataset_list = [dataset16, dataset22]

#for i in dataset_list:
    #image = dataset_list[i].read()
    #print(image)
#this did not work, because the list indices must be integers or slices, not DatasetReader

Secondly, I was wondering how I could crop the image in a way that would only load the image for the window around the mining site located at long/lat decimal -1.0333 and 27.7333. I checked the image and figured that that spatial location would correspond roughly to a quarter of the height (row) and half the width (column) of the original image. I thus tried the following code, but that does not seem to be the window/spatial location I wanted. I also tried with the coordinates directly, but that did not work. So I am wondering how one could clip/crop the window directly around the decimal coordinates. 

In [ ]:
#centeri, centerj = img16.height//4, img16.width//2 
#centerx, centery = img16.transform * (centerj, centeri) # note the reversal here, from i,j to j,i
#top, lft = img16.index(centerx-6000, centery+6000)
#bot, rgt = img16.index(centerx+6000, centery-6000)
#subset = img16.read(window=((top, bot), (lft, rgt)))

 #I tried the following, but that did not work either:

#centerx, centery = img16.index(-1.0333, 27.7333)
#top, lft = img16.index(centerx-9000, centery+9000)
#bot, rgt = img16.index(centerx+9000, centery-9000)
#subset = img16.read(window=((top, bot), (lft, rgt)))